In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('dataincode') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

### SEGUIR DOCUMENTAÇÃO
https://iceberg.apache.org/docs/nightly/spark-queries/#dataframe

In [6]:
%run ./Datasets.ipynb

+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|order_number|order_date|qty_ordered|unit_price|    status|product_id|product_line_id|  country|created_date|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|       10168|2022-01-23|          5|    98.115|  Disputed|  S10_1949|           1002|   France|  1729302850|
|       10180|2022-01-22|          1|    951.87|In Process|  S10_2016|           1002|   Norway|  1729302850|
|       10188|2022-01-21|         65|    95.202| Cancelled|  S10_4698|           1002|Australia|  1729302850|
|       10201|2022-01-26|          8|    951.17|   On Hold|  S10_4757|           1221|  Finland|  1729302850|
|       10211|2022-01-31|         34|    95.195|  Resolved|  S10_4962|           1221|  Austria|  1729302850|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
only showi

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+------------+----------+-----------+----------+----------+----------+---------------+-----------+
|order_number|order_date|qty_ordered|unit_price|    status|product_id|product_line_id|    country|
+------------+----------+-----------+----------+----------+----------+---------------+-----------+
|       12190|2023-08-23|         40|    98.115|   Shipped|  S10_1949|           1002|     Brazil|
|       12191|2023-09-22|         11|    951.87|  Resolved|  S10_2016|           1002|    Ireland|
|       12192|2023-10-04|         50|    95.202|In Process|  S10_4698|           1002|  Australia|
|       10401|2023-01-23|         10|   1047.99|   On Hold|  S18_1129|           1004|Switzerland|
+------------+----------+-----------+----------+----------+----------+---------------+-----------+



In [ ]:
orders_df.writeTo("iceberg.bronze.sales_order").createOrReplace()

In [16]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order LIMIT 5").toPandas()

,order_number,order_date,qty_ordered,unit_price,status,product_id,product_line_id,country,created_date
0,10168,2022-01-23,5,98.115,Disputed,S10_1949,1002,France,1729302860
1,10180,2022-01-22,1,951.87,In Process,S10_2016,1002,Norway,1729302860
2,10188,2022-01-21,65,95.202,Cancelled,S10_4698,1002,Australia,1729302860
3,10201,2022-01-26,8,951.17,On Hold,S10_4757,1221,Finland,1729302860
4,10211,2022-01-31,34,95.195,Resolved,S10_4962,1221,Austria,1729302860


## INSERIR NOVAS LINHAS

### ANALISAR TABELA

In [49]:
## Verificar historico
spark.sql("SELECT * FROM iceberg.bronze.sales_order.history").toPandas()


,made_current_at,snapshot_id,parent_id,is_current_ancestor
0,2024-10-19 01:44:17.005,4219981021645639449,NaN,False
1,2024-10-19 01:54:24.074,6823539219026946926,NaN,True


In [14]:
# Descrever a tabela para obter informações sobre a versão
spark.sql("DESCRIBE iceberg.bronze.sales_order").show()


+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|   order_number|   string|   NULL|
|     order_date|   string|   NULL|
|    qty_ordered|   string|   NULL|
|     unit_price|   string|   NULL|
|         status|   string|   NULL|
|     product_id|   string|   NULL|
|product_line_id|   string|   NULL|
|        country|   string|   NULL|
|   created_date|   bigint|   NULL|
+---------------+---------+-------+



## Fazer um Roll Back para versão anterior

In [48]:
spark.sql("CALL iceberg.system.rollback_to_snapshot('iceberg.bronze.sales_order', 4219981021645639449)").show()


In [25]:
## Verificar historico
spark.sql("SELECT * FROM iceberg.bronze.sales_order.history").show()

+--------------------+-------------------+---------+-------------------+
|     made_current_at|        snapshot_id|parent_id|is_current_ancestor|
+--------------------+-------------------+---------+-------------------+
|2024-10-19 01:44:...|4219981021645639449|     NULL|              false|
|2024-10-19 01:54:...|6823539219026946926|     NULL|               true|
+--------------------+-------------------+---------+-------------------+



In [28]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order TIMESTAMP AS OF '2024-10-19 01:44:17.005' ").show(5)

+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|order_number|order_date|qty_ordered|unit_price|    status|product_id|product_line_id|  country|created_date|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|       10168|2022-01-23|          5|    98.115|  Disputed|  S10_1949|           1002|   France|  1729302251|
|       10180|2022-01-22|          1|    951.87|In Process|  S10_2016|           1002|   Norway|  1729302251|
|       10188|2022-01-21|         65|    95.202| Cancelled|  S10_4698|           1002|Australia|  1729302251|
|       10201|2022-01-26|          8|    951.17|   On Hold|  S10_4757|           1221|  Finland|  1729302251|
|       10211|2022-01-31|         34|    95.195|  Resolved|  S10_4962|           1221|  Austria|  1729302251|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
only showi

In [29]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order VERSION AS OF '4219981021645639449' ").show(5)

+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|order_number|order_date|qty_ordered|unit_price|    status|product_id|product_line_id|  country|created_date|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
|       10168|2022-01-23|          5|    98.115|  Disputed|  S10_1949|           1002|   France|  1729302251|
|       10180|2022-01-22|          1|    951.87|In Process|  S10_2016|           1002|   Norway|  1729302251|
|       10188|2022-01-21|         65|    95.202| Cancelled|  S10_4698|           1002|Australia|  1729302251|
|       10201|2022-01-26|          8|    951.17|   On Hold|  S10_4757|           1221|  Finland|  1729302251|
|       10211|2022-01-31|         34|    95.195|  Resolved|  S10_4962|           1221|  Austria|  1729302251|
+------------+----------+-----------+----------+----------+----------+---------------+---------+------------+
only showi

### Mostrar arquivos da tabela

In [32]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order.files;").toPandas()

,content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2634,"{1: 56, 2: 58, 3: 48, 4: 58, 5: 64, 6: 62, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 49, 54, 56], 2: [50, 48, 50, 50, ...","{1: [49, 48, 49, 56, 48], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((58, 2, 0, None, France, Norway), (70, 2, 0, ..."
1,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2595,"{1: 56, 2: 59, 3: 49, 4: 58, 5: 62, 6: 59, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 49, 56, 56], 2: [50, 48, 50, 50, ...","{1: [49, 48, 50, 48, 49], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((62, 2, 0, None, Australia, Finland), (70, 2,..."
2,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2632,"{1: 56, 2: 62, 3: 49, 4: 58, 5: 64, 6: 61, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 50, 49, 49], 2: [50, 48, 50, 50, ...","{1: [49, 48, 50, 50, 51], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((55, 2, 0, None, Austria, UK), (70, 2, 0, Non..."
3,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,4,2764,"{1: 62, 2: 64, 3: 61, 4: 71, 5: 104, 6: 71, 7:...","{1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 50, 51, 55], 2: [50, 48, 50, 50, ...","{1: [49, 48, 50, 55, 53], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((80, 4, 0, None, Canada, Sweden), (70, 4, 0, ..."
4,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2684,"{1: 56, 2: 59, 3: 49, 4: 57, 5: 82, 6: 58, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 50, 56, 53], 2: [50, 48, 50, 50, ...","{1: [49, 48, 50, 57, 57], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((56, 2, 0, None, Italy, Japan), (70, 2, 0, No..."
5,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2595,"{1: 56, 2: 59, 3: 52, 4: 58, 5: 60, 6: 62, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 51, 48, 57], 2: [50, 48, 50, 50, ...","{1: [49, 48, 51, 49, 56], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((60, 2, 0, None, Belgium, Denmark), (70, 2, 0..."
6,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,2,2702,"{1: 56, 2: 62, 3: 49, 4: 60, 5: 79, 6: 62, 7: ...","{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 51, 50, 57], 2: [50, 48, 50, 50, ...","{1: [49, 48, 51, 52, 49], 2: [50, 48, 50, 50, ...",None,[4],None,0,"((64, 2, 0, None, Germany, Philippines), (70, ..."
7,0,s3a://datalake/iceberg/bronze/sales_order/data...,PARQUET,0,4,2890,"{1: 64, 2: 78, 3: 62, 4: 97, 5: 92, 6: 95, 7: ...","{1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: ...","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",{},"{1: [49, 48, 51, 54, 49], 2: [50, 48, 50, 50, ...","{1: [49, 48, 52, 48, 49], 2: [50, 48, 50, 51, ...",None,[4],None,0,"((106, 4, 0, None, Brazil, Switzerland), (70, ..."


## Mostrar Manifests

In [34]:
spark.sql("SELECT * FROM  iceberg.bronze.sales_order.manifests;").toPandas()

,content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,0,s3a://datalake/iceberg/bronze/sales_order/meta...,7883,0,6823539219026946926,8,0,0,0,0,0,[]


## Mostrar Partições

In [35]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order.partitions;").toPandas()

,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
0,20,8,21496,0,0,0,0,2024-10-19 01:54:24.074,6823539219026946926


In [37]:
spark.sql("SELECT * FROM iceberg.bronze.sales_order.refs;").toPandas()

,name,type,snapshot_id,max_reference_age_in_ms,min_snapshots_to_keep,max_snapshot_age_in_ms
0,main,BRANCH,6823539219026946926,NaN,NaN,NaN


In [47]:
spark.sql("show tables in bronze").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|   bronze|  nyc_taxis|      false|
|   bronze|sales_order|      false|
+---------+-----------+-----------+



In [51]:
spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis.history").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-10-14 23:25:...|3223486258126332624|               NULL|               true|
|2024-10-18 01:07:...|1281850134764776251|3223486258126332624|               true|
|2024-10-18 01:43:...|1019886448562418034|1281850134764776251|               true|
|2024-10-18 01:43:...|7297664642128699514|1019886448562418034|               true|
|2024-10-18 01:43:...|3565655088381036050|7297664642128699514|               true|
+--------------------+-------------------+-------------------+-------------------+



In [52]:
spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis").toPandas()

,vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
0,10,1000379,1.9,16.32,N
1,11,1000380,1.9,16.32,N
2,8,1000379,1.9,16.32,N
3,7,1000375,1.9,16.32,Y
4,6,1000374,8.4,42.13,Y
5,5,1000373,0.9,9.01,N
6,2,1000371,1.8,15.32,Y
7,3,1000372,2.5,22.15,N


In [57]:
spark.sql("CALL iceberg.system.rollback_to_snapshot('iceberg.bronze.nyc_taxis', 1019886448562418034)").show()

+--------------------+-------------------+
|previous_snapshot_id|current_snapshot_id|
+--------------------+-------------------+
| 3565655088381036050|1019886448562418034|
+--------------------+-------------------+



In [58]:
spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis order by 1").toPandas()

,vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
0,2,1000371,1.8,15.32,Y
1,3,1000372,2.5,22.15,N
2,5,1000373,0.9,9.01,N
3,6,1000374,8.4,42.13,Y
4,7,1000375,1.9,16.32,Y
5,8,1000379,1.9,16.32,N


In [ ]:
# Listar arquivos de dados
spark.sql("SELECT * FROM iceberg.bronze.vendas.manifests;").show()